# Assignment 1 - Evolutionary Computation

## Imports

In [2]:
import numpy as np

## Load Data

In [13]:
tsp_problem = np.loadtxt("file-tsp.txt")

## Exercise 1 - Schemata

#### Answer

## Exercise 2 - Building Block Hypothesis

#### Answer

## Exercise 3 - Selection Pressure

#### Answer

## Exercise 4 - Role of Selection in GA's

#### Answer

## Exercise 5 - Evolutionary Strategies vs. Local Search

#### Answer

## Exercise 6 - Mementic Algorithms vs Simple EAs

#### Answer

## Exercise 7 - Genetic Programming Representation

#### Answer

## Exercise 8 - Genetic Programming Behaviour

#### Answer